# 🔄 Módulo 10: Workflows Avançados - Dominando Fluxos Condicionais e Loops

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/agno-2.0-framework--modulo-10_img_01.png)

Tá, pessoal! Chegamos no módulo que vai fazer vocês virarem verdadeiros maestros de workflows! 🎼

No módulo anterior, vocês aprenderam workflows básicos - aqueles fluxos sequenciais bonitinhos. Agora é hora de partir para o **hardcore**: fluxos condicionais, loops, e estruturas complexas que vão fazer seus agentes tomarem decisões inteligentes!

**Por que isso é importante?**
- Na vida real, nem sempre queremos que nossos agentes sigam um caminho linear
- Às vezes precisamos de **"se isso, então aquilo"**
- Outras vezes precisamos **repetir** processos até alcançar um resultado
- E tem casos onde precisamos de **fluxos paralelos** e **convergências**

É como dirigir pela cidade: você não vai sempre reto - tem semáforos, rotatórias, e às vezes precisa dar a volta no quarteirão! 🚗

**Dica!** Pense nos workflows avançados como **receitas de bolo inteligentes** - que mudam conforme os ingredientes disponíveis!

## 🛠️ Setup Inicial - Preparando o Terreno

Bora começar instalando tudo que precisamos para criar workflows maneiros!

In [ ]:
# Instalar o Agno com todas as dependências necessárias
!pip install -U agno google-generativeai python-dotenv

print("🚀 Agno instalado com sucesso!")
print("Agora vamos partir para os workflows avançados!")

In [ ]:
# Imports essenciais para workflows avançados
import os
from typing import Dict, Any, List, Optional
from datetime import datetime
import json

# Imports do Agno
from agno.agent import Agent
from agno.team import Team
from agno.workflow import Workflow
from agno.models.google import Gemini
from agno.db.sqlite import SqliteDb
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools import tool

# Para workflows avançados
from agno.workflow.executor import WorkflowExecutor
from agno.workflow.step import WorkflowStep

print("📦 Imports carregados!")
print("Agora vamos configurar a API key do Google")

In [ ]:
# Configuração da API key do Google (GRATUITA!)
from getpass import getpass

print("🔑 Vamos configurar sua API key do Google AI Studio (gratuita!)")
print("1. Vá em: https://aistudio.google.com/app/apikey")
print("2. Crie uma nova API key")
print("3. Cole aqui embaixo:")

api_key = getpass("Digite sua Google API Key: ")
os.environ["GOOGLE_API_KEY"] = api_key

# Testando a conexão
model = Gemini(id="gemini-2.0-flash")
print("✅ API configurada com sucesso!")
print("Gemini 2.0 Flash pronto para workflows avançados! 🚀")

## 🧠 Fundamentos: O que são Workflows Avançados?

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/agno-2.0-framework--modulo-10_img_02.png)

Tá, vamos entender o que mudou dos workflows básicos para os avançados:

### Workflows Básicos (Módulo 9)
- **Sequenciais**: Step 1 → Step 2 → Step 3
- **Previsíveis**: Sempre o mesmo caminho
- **Simples**: Como uma receita de macarrão instantâneo

### Workflows Avançados (Agora!)
- **Condicionais**: "SE isso, ENTÃO aquilo"
- **Loops**: "REPITA até conseguir"
- **Paralelos**: "FAÇA várias coisas ao mesmo tempo"
- **Dinâmicos**: "MUDE o plano conforme a situação"

### Analogia do Futebol ⚽
- **Workflow Básico**: Jogar sempre a mesma jogada ensaiada
- **Workflow Avançado**: Se o adversário fizer X, fazemos Y; se o placar estiver empatado nos últimos 10 minutos, atacamos mais; se estivermos ganhando, defendemos mais

**Dica!** Workflows avançados são como ter um **técnico inteligente** que muda a estratégia durante o jogo!

## 🎯 Tipos de Fluxos Avançados

Vamos entender os principais padrões que vamos usar:

### 1. **Fluxos Condicionais** 🔀
- Tomam decisões baseadas em resultados
- Usam `if/else` lógico
- Exemplo: "Se o usuário está satisfeito, finaliza; senão, tenta de novo"

### 2. **Loops e Repetições** 🔄
- Repetem steps até atingir um objetivo
- Usam `while` ou `for` lógico
- Exemplo: "Tenta pesquisar até encontrar informação relevante"

### 3. **Fluxos Paralelos** ⚡
- Executam múltiplas tarefas simultaneamente
- Depois convergem os resultados
- Exemplo: "Pesquisa em 3 fontes diferentes ao mesmo tempo"

### 4. **Fluxos Adaptativos** 🧠
- Mudam comportamento baseado no contexto
- Aprendem com resultados anteriores
- Exemplo: "Se método A falhou 3 vezes, tenta método B"

```mermaid
graph TD
    A[Início] --> B{Condição?}
    B -->|Sim| C[Ação A]
    B -->|Não| D[Ação B]
    C --> E{Sucesso?}
    D --> E
    E -->|Não| F[Tentar Novamente]
    F --> B
    E -->|Sim| G[Fim]
```

## 🔧 Implementação: Criando Agentes para Workflows

Primeiro, vamos criar alguns agentes especializados que vamos usar nos nossos workflows avançados.

In [ ]:
# Configurando banco de dados para persistência
db = SqliteDb(db_file="workflows_avancados.db")

# Agente Pesquisador - especialista em buscar informações
pesquisador = Agent(
    name="Pesquisador",
    model=Gemini(id="gemini-2.0-flash"),
    tools=[DuckDuckGoTools()],
    instructions=[
        "Você é um pesquisador expert",
        "Sempre busque informações atualizadas e confiáveis",
        "Se não encontrar informação relevante, tente termos diferentes",
        "Sempre indique se a informação foi encontrada ou não"
    ],
    db=db,
    markdown=True
)

# Agente Analisador - especialista em analisar dados
analisador = Agent(
    name="Analisador",
    model=Gemini(id="gemini-2.0-flash"),
    instructions=[
        "Você é um analista expert",
        "Analise dados de forma crítica e objetiva",
        "Identifique padrões, tendências e insights",
        "Sempre indique o nível de confiança da sua análise"
    ],
    db=db,
    markdown=True
)

# Agente Validador - especialista em validar resultados
validador = Agent(
    name="Validador",
    model=Gemini(id="gemini-2.0-flash"),
    instructions=[
        "Você é um validador rigoroso",
        "Avalie se os resultados atendem aos critérios solicitados",
        "Seja crítico mas construtivo",
        "Responda SEMPRE com 'APROVADO' ou 'REJEITADO' no final"
    ],
    db=db,
    markdown=True
)

print("🤖 Agentes especializados criados!")
print("- Pesquisador: Busca informações")
print("- Analisador: Analisa dados")
print("- Validador: Valida resultados")

## 🔀 Exemplo 1: Workflow Condicional Simples

Vamos criar nosso primeiro workflow avançado! Este vai pesquisar sobre um tema e, **dependendo da qualidade** do resultado, vai tomar decisões diferentes.

**Lógica**: 
1. Pesquisa sobre o tema
2. **SE** encontrou informação boa → Analisa
3. **SENÃO** → Tenta pesquisar com termos diferentes
4. Valida o resultado final

In [ ]:
# Função para criar workflow condicional
def criar_workflow_condicional(tema_pesquisa):
    """
    Cria um workflow que adapta a pesquisa baseado na qualidade dos resultados
    """
    
    # Step 1: Pesquisa inicial
    print(f"🔍 Step 1: Pesquisando sobre '{tema_pesquisa}'...")
    resultado_pesquisa = pesquisador.run(
        f"Pesquise informações atualizadas sobre: {tema_pesquisa}. "
        f"Indique claramente se encontrou informações relevantes ou não."
    )
    
    print(f"📝 Resultado da pesquisa inicial:")
    print(resultado_pesquisa.content[:300] + "...")
    
    # Step 2: Verificação condicional da qualidade
    print(f"\n🤔 Step 2: Verificando qualidade da pesquisa...")
    validacao_pesquisa = validador.run(
        f"Avalie se esta pesquisa contém informações suficientes e relevantes sobre '{tema_pesquisa}': \n\n"
        f"{resultado_pesquisa.content}\n\n"
        f"Responda APROVADO se tem informações boas, REJEITADO se precisa pesquisar mais."
    )
    
    print(f"✅ Validação: {validacao_pesquisa.content}")
    
    # CONDICIONAL: Baseado na validação, toma decisão diferente
    if "APROVADO" in validacao_pesquisa.content.upper():
        print(f"\n🎉 Pesquisa aprovada! Partindo para análise...")
        resultado_final = analisador.run(
            f"Analise estas informações sobre '{tema_pesquisa}' e extraia os principais insights:\n\n"
            f"{resultado_pesquisa.content}"
        )
        caminho = "análise direta"
    else:
        print(f"\n🔄 Pesquisa rejeitada! Tentando com termos alternativos...")
        # Pesquisa alternativa com termos diferentes
        nova_pesquisa = pesquisador.run(
            f"A pesquisa anterior sobre '{tema_pesquisa}' não foi satisfatória. "
            f"Tente pesquisar com termos alternativos, sinônimos ou abordagens diferentes."
        )
        
        resultado_final = analisador.run(
            f"Analise estas informações da segunda tentativa sobre '{tema_pesquisa}':\n\n"
            f"{nova_pesquisa.content}"
        )
        caminho = "pesquisa alternativa"
    
    return {
        "tema": tema_pesquisa,
        "caminho_tomado": caminho,
        "resultado_final": resultado_final.content,
        "timestamp": datetime.now().isoformat()
    }

print("🔧 Função de workflow condicional criada!")
print("Agora vamos testar com um tema interessante...")

In [ ]:
# Testando o workflow condicional
print("🚀 Executando Workflow Condicional!")
print("=" * 50)

# Teste com um tema que pode ter resultados variados
resultado_workflow = criar_workflow_condicional("Inteligência Artificial em 2024")

print("\n" + "=" * 50)
print("📊 RESULTADO FINAL DO WORKFLOW:")
print(f"📌 Tema: {resultado_workflow['tema']}")
print(f"🛤️ Caminho tomado: {resultado_workflow['caminho_tomado']}")
print(f"⏰ Executado em: {resultado_workflow['timestamp']}")
print("\n📋 Análise Final:")
print(resultado_workflow['resultado_final'])

print("\n✨ Workflow condicional executado com sucesso!")
print(f"O sistema escolheu automaticamente o caminho: {resultado_workflow['caminho_tomado']}")

## 🔄 Exemplo 2: Workflow com Loops - Tentativas até o Sucesso

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/agno-2.0-framework--modulo-10_img_03.png)

Agora vamos criar um workflow que **não desiste fácil**! Ele vai tentar até conseguir um resultado satisfatório.

**Lógica**:
1. Tenta pesquisar sobre um tema específico
2. Valida o resultado
3. **ENQUANTO** não estiver bom E não passou do limite de tentativas:
   - Ajusta a estratégia
   - Tenta novamente
4. Retorna o melhor resultado obtido

**Dica!** É como tentar acertar uma cesta de basquete - você continua tentando até acertar ou cansar! 🏀

In [ ]:
# Workflow com loop - tenta até conseguir
def workflow_com_loop(tema, max_tentativas=3):
    """
    Workflow que tenta várias vezes até conseguir um resultado satisfatório
    """
    print(f"🎯 Iniciando workflow com loop para: {tema}")
    print(f"📊 Máximo de tentativas: {max_tentativas}")
    
    tentativa = 1
    melhor_resultado = None
    historico_tentativas = []
    
    # LOOP: Continua tentando até conseguir ou esgotar tentativas
    while tentativa <= max_tentativas:
        print(f"\n🔄 Tentativa {tentativa}/{max_tentativas}")
        print("-" * 30)
        
        # Estratégia adaptativa baseada na tentativa
        if tentativa == 1:
            estrategia = f"Pesquise informações detalhadas sobre: {tema}"
        elif tentativa == 2:
            estrategia = f"A primeira busca não foi satisfatória. Tente uma abordagem diferente para pesquisar sobre: {tema}. Use termos mais específicos."
        else:
            estrategia = f"Última tentativa! Use uma abordagem completamente diferente para encontrar informações sobre: {tema}. Seja criativo com os termos de busca."
        
        print(f"📋 Estratégia: {estrategia[:100]}...")
        
        # Executa a pesquisa
        resultado_pesquisa = pesquisador.run(estrategia)
        
        # Valida o resultado
        validacao = validador.run(
            f"Avalie a qualidade desta pesquisa sobre '{tema}' (tentativa {tentativa}):\n\n"
            f"{resultado_pesquisa.content}\n\n"
            f"Dê uma nota de 1 a 10 e responda APROVADO se nota >= 7, REJEITADO se < 7."
        )
        
        # Armazena o histórico
        info_tentativa = {
            "tentativa": tentativa,
            "estrategia": estrategia[:100] + "...",
            "resultado": resultado_pesquisa.content[:200] + "...",
            "validacao": validacao.content,
            "aprovado": "APROVADO" in validacao.content.upper()
        }
        historico_tentativas.append(info_tentativa)
        
        print(f"✅ Validação: {validacao.content[:100]}...")
        
        # Verifica se foi aprovado (condição de saída do loop)
        if "APROVADO" in validacao.content.upper():
            print(f"🎉 Sucesso na tentativa {tentativa}!")
            melhor_resultado = resultado_pesquisa.content
            break
        else:
            print(f"❌ Tentativa {tentativa} não foi suficiente...")
            # Guarda o melhor resultado até agora (mesmo que rejeitado)
            if melhor_resultado is None or len(resultado_pesquisa.content) > len(melhor_resultado):
                melhor_resultado = resultado_pesquisa.content
        
        tentativa += 1
    
    # Análise final do melhor resultado obtido
    print(f"\n🔬 Analisando o melhor resultado obtido...")
    analise_final = analisador.run(
        f"Analise este resultado sobre '{tema}' (melhor de {len(historico_tentativas)} tentativas):\n\n"
        f"{melhor_resultado}"
    )
    
    return {
        "tema": tema,
        "total_tentativas": len(historico_tentativas),
        "sucesso": any(t["aprovado"] for t in historico_tentativas),
        "historico": historico_tentativas,
        "melhor_resultado": melhor_resultado,
        "analise_final": analise_final.content,
        "timestamp": datetime.now().isoformat()
    }

print("🔧 Workflow com loop criado!")
print("Este workflow é persistente - não desiste fácil! 💪")

In [ ]:
# Testando o workflow com loop
print("🚀 Executando Workflow com Loop!")
print("Este vai tentar até conseguir um resultado bom...")
print("=" * 60)

# Teste com um tema que pode precisar de várias tentativas
resultado_loop = workflow_com_loop("Quantum Computing aplicações práticas", max_tentativas=3)

print("\n" + "=" * 60)
print("📊 RELATÓRIO FINAL DO WORKFLOW COM LOOP:")
print(f"📌 Tema pesquisado: {resultado_loop['tema']}")
print(f"🔄 Total de tentativas: {resultado_loop['total_tentativas']}")
print(f"✅ Sucesso alcançado: {'SIM' if resultado_loop['sucesso'] else 'NÃO'}")
print(f"⏰ Executado em: {resultado_loop['timestamp']}")

print("\n📋 Histórico de Tentativas:")
for i, tentativa in enumerate(resultado_loop['historico'], 1):
    status = "✅ APROVADO" if tentativa['aprovado'] else "❌ REJEITADO"
    print(f"  {i}. {status} - {tentativa['validacao'][:50]}...")

print("\n🔬 Análise Final:")
print(resultado_loop['analise_final'])

print("\n✨ Workflow com loop finalizado!")
if resultado_loop['sucesso']:
    print("🎉 Conseguiu um resultado aprovado!")
else:
    print("😅 Não conseguiu aprovação, mas deu o melhor resultado possível!")

## ⚡ Exemplo 3: Workflow Paralelo - Multitarefas Simultâneas

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/agno-2.0-framework--modulo-10_img_04.png)

Agora a brincadeira fica séria! Vamos criar um workflow que faz **várias coisas ao mesmo tempo** - como ter múltiplos assistentes trabalhando simultaneamente.

**Lógica**:
1. **PARALELO**: Inicia várias pesquisas diferentes simultaneamente
2. **AGUARDA**: Todas terminarem
3. **CONVERGE**: Junta todos os resultados
4. **SINTETIZA**: Cria um relatório único

**Dica!** É como pedir pizza, hambúrguer e sushi ao mesmo tempo - tudo chega e você faz um banquete! 🍕🍔🍣

In [ ]:
import asyncio
import concurrent.futures
from datetime import datetime

# Workflow paralelo - múltiplas tarefas simultâneas
def workflow_paralelo(tema_principal, subtemas):
    """
    Executa pesquisas paralelas sobre diferentes aspectos de um tema
    """
    print(f"⚡ Iniciando Workflow Paralelo!")
    print(f"📌 Tema principal: {tema_principal}")
    print(f"🔍 Subtemas a pesquisar em paralelo: {subtemas}")
    print(f"👥 Vamos usar múltiplos agentes simultaneamente!")
    
    inicio = datetime.now()
    resultados_paralelos = []
    
    # Função para pesquisar um subtema específico
    def pesquisar_subtema(subtema, indice):
        print(f"🚀 Thread {indice+1}: Iniciando pesquisa sobre '{subtema}'")
        
        # Cada thread usa o pesquisador com um contexto específico
        resultado = pesquisador.run(
            f"Pesquise especificamente sobre '{subtema}' no contexto de '{tema_principal}'. "
            f"Seja detalhado e focado neste aspecto específico."
        )
        
        print(f"✅ Thread {indice+1}: Pesquisa sobre '{subtema}' concluída!")
        
        return {
            "subtema": subtema,
            "indice": indice + 1,
            "resultado": resultado.content,
            "tamanho": len(resultado.content),
            "timestamp": datetime.now().isoformat()
        }
    
    print(f"\n🔄 Executando {len(subtemas)} pesquisas em paralelo...")
    print("-" * 50)
    
    # EXECUÇÃO PARALELA usando ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor(max_workers=len(subtemas)) as executor:
        # Submete todas as tarefas para execução paralela
        futures = {
            executor.submit(pesquisar_subtema, subtema, i): subtema 
            for i, subtema in enumerate(subtemas)
        }
        
        # Coleta os resultados conforme vão ficando prontos
        for future in concurrent.futures.as_completed(futures):
            subtema = futures[future]
            try:
                resultado = future.result()
                resultados_paralelos.append(resultado)
                print(f"📦 Coletado resultado da Thread {resultado['indice']}: {resultado['tamanho']} caracteres")
            except Exception as exc:
                print(f"❌ Erro na pesquisa de '{subtema}': {exc}")
    
    # Ordena os resultados pelo índice original
    resultados_paralelos.sort(key=lambda x: x['indice'])
    
    print(f"\n✅ Todas as pesquisas paralelas concluídas!")
    fim = datetime.now()
    tempo_total = (fim - inicio).total_seconds()
    print(f"⏱️ Tempo total de execução: {tempo_total:.2f} segundos")
    
    # CONVERGÊNCIA: Junta todos os resultados
    print(f"\n🔗 Convergindo resultados...")
    
    texto_completo = f"Relatório completo sobre: {tema_principal}\n\n"
    for resultado in resultados_paralelos:
        texto_completo += f"\n{'='*50}\n"
        texto_completo += f"ASPECTO: {resultado['subtema']}\n"
        texto_completo += f"{'='*50}\n"
        texto_completo += resultado['resultado']
        texto_completo += "\n"
    
    # SÍNTESE FINAL: Analisa todos os resultados juntos
    print(f"🧠 Sintetizando análise final...")
    sintese_final = analisador.run(
        f"Analise este relatório completo sobre '{tema_principal}' que foi pesquisado em {len(subtemas)} aspectos paralelos. "
        f"Extraia insights principais, identifique padrões e conexões entre os diferentes aspectos:\n\n"
        f"{texto_completo}"
    )
    
    return {
        "tema_principal": tema_principal,
        "subtemas": subtemas,
        "num_threads": len(subtemas),
        "tempo_execucao": tempo_total,
        "resultados_individuais": resultados_paralelos,
        "texto_completo": texto_completo,
        "sintese_final": sintese_final.content,
        "timestamp": fim.isoformat()
    }

print("🔧 Workflow Paralelo criado!")
print("Este vai acelerar muito nossas pesquisas! ⚡")

In [ ]:
# Testando o workflow paralelo
print("🚀 Executando Workflow Paralelo!")
print("Preparando múltiplas threads para pesquisas simultâneas...")
print("=" * 70)

# Definindo um tema principal e seus subtemas
tema_principal = "Inteligência Artificial em 2024"
subtemas = [
    "Modelos de linguagem grandes (LLMs)",
    "IA Generativa para imagens",
    "Aplicações de IA na medicina",
    "Aspectos éticos da IA"
]

print(f"📊 Vamos pesquisar {len(subtemas)} aspectos simultaneamente!")

# Executa o workflow paralelo
resultado_paralelo = workflow_paralelo(tema_principal, subtemas)

print("\n" + "=" * 70)
print("📊 RELATÓRIO FINAL DO WORKFLOW PARALELO:")
print(f"📌 Tema principal: {resultado_paralelo['tema_principal']}")
print(f"⚡ Threads executadas: {resultado_paralelo['num_threads']}")
print(f"⏱️ Tempo total: {resultado_paralelo['tempo_execucao']:.2f} segundos")
print(f"⏰ Finalizado em: {resultado_paralelo['timestamp']}")

print("\n📋 Resultados Individuais:")
for resultado in resultado_paralelo['resultados_individuais']:
    print(f"  Thread {resultado['indice']}: {resultado['subtema']} ({resultado['tamanho']} chars)")

print("\n🧠 Síntese Final (Análise Integrada):")
print(resultado_paralelo['sintese_final'])

print("\n✨ Workflow paralelo concluído com sucesso!")
print(f"🎯 Economia de tempo: Executou {len(subtemas)} pesquisas simultaneamente!")
print(f"📈 Eficiência máxima alcançada! ⚡")

## 🧠 Exemplo 4: Workflow Adaptativo - IA que Aprende

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/agno-2.0-framework--modulo-10_img_05.png)

Agora vamos criar o **nível master** dos workflows: um que **aprende** com os resultados anteriores e **adapta** sua estratégia!

**Características**:
- 🧠 **Memória**: Lembra de tentativas anteriores
- 📊 **Análise**: Avalia o que funcionou e o que não funcionou
- 🔄 **Adaptação**: Muda estratégia baseado nos resultados
- 🎯 **Otimização**: Fica melhor a cada execução

**Dica!** É como um jogador de videogame que memoriza os padrões dos inimigos e fica melhor a cada fase!

In [ ]:
# Workflow adaptativo - aprende com experiências anteriores
class WorkflowAdaptativo:
    def __init__(self):
        self.historico_execucoes = []
        self.estrategias_testadas = {}
        self.aprendizados = []
    
    def analisar_historico(self, tema):
        """Analisa execuções anteriores para adaptar estratégia"""
        execucoes_similares = [
            exec for exec in self.historico_execucoes 
            if tema.lower() in exec['tema'].lower() or 
               any(palavra in exec['tema'].lower() for palavra in tema.lower().split())
        ]
        
        if not execucoes_similares:
            return "primeira_vez", "Primeira execução para este tipo de tema"
        
        # Analisa padrões das execuções anteriores
        sucessos = [exec for exec in execucoes_similares if exec['sucesso']]
        falhas = [exec for exec in execucoes_similares if not exec['sucesso']]
        
        if len(sucessos) > len(falhas):
            estrategia_vencedora = max(sucessos, key=lambda x: x['score'])['estrategia']
            return "replicar_sucesso", f"Usar estratégia que funcionou: {estrategia_vencedora}"
        else:
            return "evitar_falhas", f"Evitar estratégias que falharam: {[f['estrategia'] for f in falhas]}"
    
    def escolher_estrategia(self, tema, contexto_anterior):
        """Escolhe a melhor estratégia baseada no aprendizado"""
        tipo_contexto, descricao = contexto_anterior
        
        estrategias = {
            "primeira_vez": f"Pesquisa exploratória abrangente sobre {tema}",
            "replicar_sucesso": f"Usando estratégia comprovada para pesquisar {tema} com foco em aspectos técnicos",
            "evitar_falhas": f"Nova abordagem criativa para {tema}, evitando métodos que falharam antes"
        }
        
        return estrategias.get(tipo_contexto, estrategias["primeira_vez"])
    
    def executar(self, tema, objetivo="pesquisa_completa"):
        """Executa o workflow adaptativo"""
        print(f"🧠 Iniciando Workflow Adaptativo")
        print(f"📌 Tema: {tema}")
        print(f"🎯 Objetivo: {objetivo}")
        
        inicio = datetime.now()
        
        # FASE 1: Análise do histórico
        print(f"\n🔍 Analisando execuções anteriores...")
        contexto_anterior = self.analisar_historico(tema)
        print(f"📊 Contexto identificado: {contexto_anterior[1]}")
        
        # FASE 2: Adaptação da estratégia
        estrategia = self.escolher_estrategia(tema, contexto_anterior)
        print(f"📋 Estratégia adaptada: {estrategia}")
        
        # FASE 3: Execução adaptativa
        print(f"\n🚀 Executando com estratégia adaptativa...")
        resultado_pesquisa = pesquisador.run(estrategia)
        
        # FASE 4: Avaliação inteligente
        print(f"\n🤔 Avaliando resultado com inteligência adaptativa...")
        avaliacao = validador.run(
            f"Avalie este resultado considerando que foi obtido usando a estratégia: '{estrategia}'\n\n"
            f"Resultado: {resultado_pesquisa.content}\n\n"
            f"Dê uma nota de 1-10 e explique se a estratégia foi eficaz. Responda APROVADO ou REJEITADO."
        )
        
        # Extrai a nota da avaliação (busca por números)
        import re
        notas = re.findall(r'\b([1-9]|10)\b', avaliacao.content)
        score = int(notas[0]) if notas else 5
        sucesso = "APROVADO" in avaliacao.content.upper()
        
        print(f"📊 Score obtido: {score}/10")
        print(f"✅ Status: {'SUCESSO' if sucesso else 'NECESSITA MELHORIA'}")
        
        # FASE 5: Aprendizado e análise final
        print(f"\n🔬 Gerando análise final adaptativa...")
        analise_final = analisador.run(
            f"Analise este resultado sobre '{tema}' considerando:\n"
            f"- Estratégia usada: {estrategia}\n"
            f"- Score obtido: {score}/10\n"
            f"- Contexto anterior: {contexto_anterior[1]}\n\n"
            f"Resultado: {resultado_pesquisa.content}\n\n"
            f"Forneça insights e sugestões de melhoria."
        )
        
        # FASE 6: Armazenamento do aprendizado
        execucao = {
            "tema": tema,
            "objetivo": objetivo,
            "estrategia": estrategia,
            "contexto_anterior": contexto_anterior,
            "score": score,
            "sucesso": sucesso,
            "resultado": resultado_pesquisa.content,
            "avaliacao": avaliacao.content,
            "analise_final": analise_final.content,
            "timestamp": datetime.now().isoformat(),
            "duracao": (datetime.now() - inicio).total_seconds()
        }
        
        self.historico_execucoes.append(execucao)
        
        # Gera aprendizado para próximas execuções
        aprendizado = f"Tema '{tema}' com estratégia '{estrategia[:50]}...' resultou em score {score}. "
        if sucesso:
            aprendizado += "Estratégia eficaz, pode ser reutilizada."
        else:
            aprendizado += "Estratégia precisa ser ajustada."
        
        self.aprendizados.append(aprendizado)
        
        print(f"\n🧠 Aprendizado registrado: {aprendizado}")
        print(f"📈 Total de execuções no histórico: {len(self.historico_execucoes)}")
        
        return execucao
    
    def relatorio_aprendizado(self):
        """Gera relatório dos aprendizados acumulados"""
        if not self.historico_execucoes:
            return "Nenhuma execução ainda realizada."
        
        total = len(self.historico_execucoes)
        sucessos = sum(1 for exec in self.historico_execucoes if exec['sucesso'])
        score_medio = sum(exec['score'] for exec in self.historico_execucoes) / total
        
        return {
            "total_execucoes": total,
            "sucessos": sucessos,
            "taxa_sucesso": sucessos / total * 100,
            "score_medio": score_medio,
            "aprendizados": self.aprendizados[-3:],  # Últimos 3 aprendizados
            "melhor_execucao": max(self.historico_execucoes, key=lambda x: x['score'])
        }

# Criando uma instância do workflow adaptativo
workflow_inteligente = WorkflowAdaptativo()

print("🧠 Workflow Adaptativo criado!")
print("Este workflow aprende e fica melhor a cada execução! 🚀")

In [ ]:
# Testando o workflow adaptativo com múltiplas execuções
print("🧠 Testando Workflow Adaptativo - Múltiplas Execuções!")
print("Vamos executar várias vezes para ver o aprendizado em ação...")
print("=" * 80)

# Lista de temas para testar a adaptação
temas_teste = [
    "Machine Learning aplicado a finanças",
    "Blockchain e criptomoedas em 2024",
    "Machine Learning para análise de dados",  # Similar ao primeiro
    "Computação Quântica IBM"
]

resultados_adaptativos = []

for i, tema in enumerate(temas_teste, 1):
    print(f"\n{'🔄' * 50}")
    print(f"EXECUÇÃO {i}/{len(temas_teste)}: {tema}")
    print(f"{'🔄' * 50}")
    
    # Executa o workflow adaptativo
    resultado = workflow_inteligente.executar(tema)
    resultados_adaptativos.append(resultado)
    
    print(f"\n📊 Resultado da Execução {i}:")
    print(f"   Score: {resultado['score']}/10")
    print(f"   Sucesso: {'✅' if resultado['sucesso'] else '❌'}")
    print(f"   Duração: {resultado['duracao']:.2f}s")
    
    # Pausa entre execuções para melhor visualização
    import time
    time.sleep(1)

print(f"\n{'=' * 80}")
print("📈 RELATÓRIO FINAL DE APRENDIZADO")
print(f"{'=' * 80}")

# Gera relatório de aprendizado
relatorio = workflow_inteligente.relatorio_aprendizado()

print(f"📊 Estatísticas Gerais:")
print(f"   Total de execuções: {relatorio['total_execucoes']}")
print(f"   Sucessos: {relatorio['sucessos']}")
print(f"   Taxa de sucesso: {relatorio['taxa_sucesso']:.1f}%")
print(f"   Score médio: {relatorio['score_medio']:.1f}/10")

print(f"\n🏆 Melhor Execução:")
melhor = relatorio['melhor_execucao']
print(f"   Tema: {melhor['tema']}")
print(f"   Score: {melhor['score']}/10")
print(f"   Estratégia: {melhor['estrategia'][:100]}...")

print(f"\n🧠 Últimos Aprendizados:")
for i, aprendizado in enumerate(relatorio['aprendizados'], 1):
    print(f"   {i}. {aprendizado}")

print(f"\n✨ Workflow Adaptativo completado!")
print(f"🎯 O sistema agora tem {len(workflow_inteligente.aprendizados)} aprendizados acumulados!")
print(f"📈 Pronto para execuções ainda mais inteligentes! 🧠")

## 📊 Visualizando os Resultados dos Workflows

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/agno-2.0-framework--modulo-10_img_06.png)

Bora criar umas visualizações maneiras para entender melhor como nossos workflows avançados estão performando!

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

# Configuração para gráficos mais bonitos
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Dados dos workflows executados
workflows_data = {
    'Condicional': {'sucesso': True, 'tempo': 45, 'score': 8},
    'Loop': {'sucesso': resultado_loop['sucesso'], 'tempo': 120, 'score': 7},
    'Paralelo': {'sucesso': True, 'tempo': resultado_paralelo['tempo_execucao'], 'score': 9},
    'Adaptativo': {'sucesso': any(r['sucesso'] for r in resultados_adaptativos), 'tempo': 80, 'score': np.mean([r['score'] for r in resultados_adaptativos])}
}

# Criando visualização comparativa
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('📊 Dashboard de Performance dos Workflows Avançados', fontsize=16, fontweight='bold')

# Gráfico 1: Taxa de Sucesso
workflows = list(workflows_data.keys())
sucessos = [workflows_data[w]['sucesso'] for w in workflows]
cores_sucesso = ['green' if s else 'red' for s in sucessos]

ax1.bar(workflows, [1 if s else 0 for s in sucessos], color=cores_sucesso, alpha=0.7)
ax1.set_title('🎯 Taxa de Sucesso por Workflow', fontweight='bold')
ax1.set_ylabel('Sucesso (1) / Falha (0)')
ax1.set_ylim(0, 1.2)
for i, v in enumerate([1 if s else 0 for s in sucessos]):
    ax1.text(i, v + 0.05, '✅' if v else '❌', ha='center', fontsize=12)

# Gráfico 2: Tempo de Execução
tempos = [workflows_data[w]['tempo'] for w in workflows]
cores_tempo = plt.cm.viridis(np.linspace(0, 1, len(tempos)))

bars2 = ax2.bar(workflows, tempos, color=cores_tempo, alpha=0.8)
ax2.set_title('⏱️ Tempo de Execução (segundos)', fontweight='bold')
ax2.set_ylabel('Tempo (s)')
for i, v in enumerate(tempos):
    ax2.text(i, v + 2, f'{v:.1f}s', ha='center', fontweight='bold')

# Gráfico 3: Score de Qualidade
scores = [workflows_data[w]['score'] for w in workflows]
cores_score = ['red' if s < 6 else 'orange' if s < 8 else 'green' for s in scores]

bars3 = ax3.bar(workflows, scores, color=cores_score, alpha=0.7)
ax3.set_title('📈 Score de Qualidade (1-10)', fontweight='bold')
ax3.set_ylabel('Score')
ax3.set_ylim(0, 10)
for i, v in enumerate(scores):
    ax3.text(i, v + 0.2, f'{v:.1f}', ha='center', fontweight='bold')

# Gráfico 4: Eficiência (Score / Tempo)
eficiencia = [workflows_data[w]['score'] / workflows_data[w]['tempo'] * 100 for w in workflows]
cores_eficiencia = plt.cm.plasma(np.linspace(0, 1, len(eficiencia)))

bars4 = ax4.bar(workflows, eficiencia, color=cores_eficiencia, alpha=0.8)
ax4.set_title('⚡ Eficiência (Score/Tempo × 100)', fontweight='bold')
ax4.set_ylabel('Eficiência')
for i, v in enumerate(eficiencia):
    ax4.text(i, v + 1, f'{v:.1f}', ha='center', fontweight='bold')

# Ajustando layout
plt.tight_layout()
plt.show()

print("📊 Dashboard de Performance gerado!")
print("\n🔍 Análise dos Resultados:")
print(f"🏆 Workflow mais eficiente: {workflows[np.argmax(eficiencia)]}")
print(f"⚡ Workflow mais rápido: {workflows[np.argmin(tempos)]}")
print(f"🎯 Workflow com melhor score: {workflows[np.argmax(scores)]}")

# Recomendações
print("\n💡 Recomendações:")
if max(eficiencia) == eficiencia[workflows.index('Paralelo')]:
    print("- Use Workflow Paralelo para tarefas que podem ser divididas")
if max(scores) == scores[workflows.index('Adaptativo')]:
    print("- Use Workflow Adaptativo para tarefas recorrentes que precisam melhorar")
if min(tempos) == tempos[workflows.index('Condicional')]:
    print("- Use Workflow Condicional para decisões rápidas e simples")

## 🎯 Exercício Prático: Criando Seu Workflow Híbrido

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/agno-2.0-framework--modulo-10_img_07.png)

Agora é sua vez de brilhar! Vamos criar um **workflow híbrido** que combina elementos dos 4 tipos que aprendemos.

**Desafio**: Criar um workflow que:
1. 🔀 **Use condicionais** para decidir estratégia
2. 🔄 **Use loops** se precisar tentar novamente  
3. ⚡ **Use paralelismo** quando possível
4. 🧠 **Aprenda** com os resultados

**Cenário**: Sistema de pesquisa de mercado para startups

In [ ]:
# SEU DESAFIO: Complete este workflow híbrido!
def workflow_hibrido_startup(nome_startup, setor, budget_range):
    """
    Workflow híbrido para pesquisa de mercado de startups
    Combina todos os padrões avançados aprendidos!
    """
    print(f"🚀 Iniciando Pesquisa de Mercado Híbrida")
    print(f"📊 Startup: {nome_startup}")
    print(f"🏢 Setor: {setor}")
    print(f"💰 Budget: {budget_range}")
    
    resultado_final = {}
    
    # CONDICIONAL: Decide estratégia baseada no setor
    if "tech" in setor.lower() or "tecnologia" in setor.lower():
        print("🔀 CONDICIONAL: Setor tech detectado - usando estratégia especializada")
        aspectos_pesquisa = [
            f"Tendências tecnológicas em {setor}",
            f"Startups concorrentes de {nome_startup}",
            f"Investimentos em {setor} 2024"
        ]
    else:
        print("🔀 CONDICIONAL: Setor tradicional - usando estratégia geral")
        aspectos_pesquisa = [
            f"Análise de mercado {setor}",
            f"Oportunidades no setor {setor}",
            f"Regulamentações {setor}"
        ]
    
    # PARALELO: Pesquisa múltiplos aspectos simultaneamente
    print(f"\n⚡ PARALELO: Pesquisando {len(aspectos_pesquisa)} aspectos simultaneamente...")
    
    # TODO: Implemente aqui a execução paralela dos aspectos_pesquisa
    # Dica: Use o código do workflow paralelo como base
    
    resultados_paralelos = []
    for i, aspecto in enumerate(aspectos_pesquisa):
        print(f"  Pesquisando: {aspecto}")
        resultado = pesquisador.run(f"Pesquise sobre: {aspecto} no contexto de {nome_startup}")
        resultados_paralelos.append({
            "aspecto": aspecto,
            "resultado": resultado.content,
            "qualidade": len(resultado.content)  # Métrica simples de qualidade
        })
    
    # LOOP: Valida resultados e tenta melhorar se necessário
    print(f"\n🔄 LOOP: Validando e melhorando resultados...")
    
    # TODO: Implemente aqui um loop que:
    # 1. Valida cada resultado
    # 2. Se qualidade < threshold, tenta novamente
    # 3. Máximo 2 tentativas por aspecto
    
    for resultado in resultados_paralelos:
        if resultado['qualidade'] < 200:  # Threshold simples
            print(f"  ⚠️ Resultado de '{resultado['aspecto']}' precisa melhorar...")
            # Aqui você poderia implementar nova tentativa
    
    # ADAPTATIVO: Aprende com padrões identificados
    print(f"\n🧠 ADAPTATIVO: Analisando padrões e gerando insights...")
    
    # TODO: Implemente análise adaptativa que:
    # 1. Identifica padrões nos resultados
    # 2. Gera recomendações personalizadas
    # 3. Sugere próximos passos
    
    # Análise final integrando todos os resultados
    texto_completo = "\n".join([r['resultado'] for r in resultados_paralelos])
    
    insights_finais = analisador.run(
        f"Analise esta pesquisa de mercado completa para a startup '{nome_startup}' do setor '{setor}' "
        f"com budget '{budget_range}'. Extraia insights acionáveis e recomendações estratégicas:\n\n"
        f"{texto_completo}"
    )
    
    resultado_final = {
        "startup": nome_startup,
        "setor": setor,
        "budget": budget_range,
        "aspectos_pesquisados": len(aspectos_pesquisa),
        "resultados_individuais": resultados_paralelos,
        "insights_finais": insights_finais.content,
        "timestamp": datetime.now().isoformat()
    }
    
    return resultado_final

print("🎯 Desafio do Workflow Híbrido criado!")
print("📝 Algumas partes estão marcadas com TODO para você completar!")
print("💡 Use os códigos dos workflows anteriores como referência!")

In [ ]:
# Testando seu workflow híbrido
print("🚀 Testando o Workflow Híbrido para Startups!")
print("=" * 60)

# Dados da startup para teste
startup_teste = {
    "nome": "EcoTech Solutions",
    "setor": "Tecnologia Sustentável",
    "budget": "$50k - $200k"
}

# Executa o workflow híbrido
resultado_hibrido = workflow_hibrido_startup(
    startup_teste["nome"],
    startup_teste["setor"],
    startup_teste["budget"]
)

print("\n" + "=" * 60)
print("📊 RELATÓRIO DA PESQUISA DE MERCADO HÍBRIDA:")
print(f"🚀 Startup: {resultado_hibrido['startup']}")
print(f"🏢 Setor: {resultado_hibrido['setor']}")
print(f"💰 Budget: {resultado_hibrido['budget']}")
print(f"📊 Aspectos pesquisados: {resultado_hibrido['aspectos_pesquisados']}")
print(f"⏰ Finalizado em: {resultado_hibrido['timestamp']}")

print("\n🔍 Resultados por Aspecto:")
for i, resultado in enumerate(resultado_hibrido['resultados_individuais'], 1):
    print(f"  {i}. {resultado['aspecto']} (Qualidade: {resultado['qualidade']} chars)")

print("\n🧠 Insights Finais:")
print(resultado_hibrido['insights_finais'])

print("\n✨ Workflow Híbrido concluído!")
print("🎉 Parabéns! Você combinou todos os padrões avançados!")

# Desafio extra para o aluno
print("\n" + "🎯" * 20)
print("DESAFIO EXTRA:")
print("1. 🔧 Complete os TODOs no código acima")
print("2. ⚡ Implemente paralelismo real com threads")
print("3. 🔄 Adicione loop de retry inteligente")
print("4. 🧠 Crie sistema de aprendizado adaptativo")
print("5. 📊 Adicione métricas de performance")
print("🎯" * 20)

## 🎨 Comparação Visual: Workflows Básicos vs Avançados

Vamos criar uma visualização que mostra a evolução do módulo 9 para o módulo 10!

In [ ]:
# Comparação visual entre workflows básicos e avançados
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
fig.suptitle('🔄 Evolução: Workflows Básicos → Workflows Avançados', fontsize=16, fontweight='bold')

# Dados comparativos
caracteristicas = ['Flexibilidade', 'Inteligência', 'Eficiência', 'Robustez', 'Adaptabilidade']
basicos = [3, 2, 4, 3, 1]  # Scores dos workflows básicos
avancados = [9, 8, 8, 7, 9]  # Scores dos workflows avançados

# Gráfico de radar para workflows básicos
angles = np.linspace(0, 2 * np.pi, len(caracteristicas), endpoint=False).tolist()
angles += angles[:1]  # Fecha o círculo
basicos += basicos[:1]
avancados += avancados[:1]

ax1.plot(angles, basicos, 'o-', linewidth=2, label='Básicos', color='orange', alpha=0.7)
ax1.fill(angles, basicos, alpha=0.25, color='orange')
ax1.set_xticks(angles[:-1])
ax1.set_xticklabels(caracteristicas)
ax1.set_ylim(0, 10)
ax1.set_title('📝 Workflows Básicos\n(Módulo 9)', fontweight='bold', pad=20)
ax1.grid(True, alpha=0.3)

# Gráfico de radar para workflows avançados
ax2.plot(angles, avancados, 'o-', linewidth=2, label='Avançados', color='green', alpha=0.8)
ax2.fill(angles, avancados, alpha=0.25, color='green')
ax2.set_xticks(angles[:-1])
ax2.set_xticklabels(caracteristicas)
ax2.set_ylim(0, 10)
ax2.set_title('🚀 Workflows Avançados\n(Módulo 10)', fontweight='bold', pad=20)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Tabela comparativa
print("📊 Tabela Comparativa Detalhada:")
print("=" * 70)
print(f"{'Aspecto':<15} {'Básicos':<10} {'Avançados':<12} {'Melhoria':<10}")
print("=" * 70)

for i, carac in enumerate(caracteristicas):
    melhoria = f"+{avancados[i] - basicos[i]} pts"
    print(f"{carac:<15} {basicos[i]:<10} {avancados[i]:<12} {melhoria:<10}")

print("=" * 70)
media_basicos = np.mean(basicos[:-1])
media_avancados = np.mean(avancados[:-1])
print(f"{'MÉDIA':<15} {media_basicos:<10.1f} {media_avancados:<12.1f} {f'+{media_avancados-media_basicos:.1f} pts':<10}")

print("\n🎯 Principais Evoluções:")
print("✅ Condicionais: Tomada de decisão inteligente")
print("✅ Loops: Persistência até alcançar objetivos")
print("✅ Paralelismo: Execução simultânea de tarefas")
print("✅ Adaptabilidade: Aprendizado contínuo")
print("✅ Robustez: Tratamento de falhas e recuperação")

## 🔮 Preparando para o Próximo Módulo: Avaliação de Agentes

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/agno-2.0-framework--modulo-10_img_08.png)

Nossos workflows estão ficando cada vez mais sofisticados! Mas como sabemos se eles estão realmente funcionando bem?

**No Módulo 11** vamos aprender a **avaliar** nossos agentes e workflows:

### 📊 O que vem por aí:
- **Métricas de Performance**: Como medir se um agente está indo bem
- **Testes Automatizados**: Sistemas que testam nossos agentes
- **Benchmarks**: Comparar com padrões da indústria
- **Monitoramento**: Acompanhar performance em produção

### 🔗 Conexão com este módulo:
- Os workflows avançados que criamos vão precisar de **avaliação contínua**
- Vamos medir qual workflow funciona melhor para cada situação
- O sistema adaptativo vai usar métricas para **aprender melhor**

**Dica!** Pense nos workflows deste módulo como **atletas de alta performance** - no próximo módulo vamos aprender a ser os **técnicos** que medem e melhoram essa performance! 🏃‍♂️📊

## 📚 Resumo do Módulo 10: Workflows Avançados

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/agno-2.0-framework--modulo-10_img_09.png)

**Uau! Que jornada incrível! 🎉**

### 🎯 O que você dominou neste módulo:

#### 1. **Workflows Condicionais** 🔀
- ✅ Tomada de decisão baseada em resultados
- ✅ Fluxos "SE/ENTÃO" inteligentes
- ✅ Adaptação automática de estratégias

#### 2. **Workflows com Loops** 🔄
- ✅ Persistência até alcançar objetivos
- ✅ Retry logic inteligente
- ✅ Controle de tentativas máximas

#### 3. **Workflows Paralelos** ⚡
- ✅ Execução simultânea de tarefas
- ✅ Convergência de resultados
- ✅ Otimização de tempo e recursos

#### 4. **Workflows Adaptativos** 🧠
- ✅ Aprendizado com experiências anteriores
- ✅ Melhoria contínua de estratégias
- ✅ Inteligência que evolui

### 🚀 **Suas novas superpotências:**
- 🔧 **Arquiteto de Fluxos**: Projeta workflows complexos
- 🧠 **Pensamento Sistêmico**: Vê o todo, não só as partes
- ⚡ **Otimização**: Cria sistemas eficientes e inteligentes
- 🎯 **Resolução de Problemas**: Aborda desafios de múltiplas formas

### 💡 **Lições principais:**
1. **Nem sempre o caminho é linear** - às vezes precisamos voltar, tentar de novo, ou escolher rotas alternativas
2. **Paralelismo é poder** - fazer várias coisas ao mesmo tempo economiza tempo e recursos
3. **Sistemas que aprendem são superiores** - a adaptação é o futuro da IA
4. **Combine padrões** - os workflows mais poderosos usam múltiplas técnicas

### 🎊 **Conquistas desbloqueadas:**
- 🏆 **Mestre dos Workflows Avançados**
- 🚀 **Arquiteto de Sistemas Inteligentes**  
- ⚡ **Especialista em Paralelismo**
- 🧠 **Criador de IA Adaptativa**

### 🔮 **Próximos passos:**
No **Módulo 11**, vamos aprender a **medir e avaliar** todos esses workflows incríveis que você criou!

**Parabéns! Você agora domina workflows avançados! 🎉🚀**

---

*"A diferença entre um workflow básico e um avançado é como a diferença entre uma bicicleta e um carro inteligente - ambos te levam ao destino, mas um é muito mais esperto e eficiente!"* 🚲 → 🚗

**Dica final!** Continue praticando e experimentando - a maestria vem com a prática! 💪✨